# Homework 2 | Exploring Options | Tasks 1-4 due 09/22/25 | Task 5 due 09/24/25

Now that you have all of the options data stored locally on your computer and in pkl files, we can begin working with it. Your task will be to edit the monthly data and make some observations about its structure. See below.

## Tasks

### 1. For each file, remove the following columns: ['ImpliedVolatility', 'Delta','Gamma', 'Vega', 'Theta']

### 2. Add a column for the SPX index. MAKE SURE THE DATES MATCH! Hint: You might want to set the index of the options data to the date column... If yFinance doesn't let you download the SPX data skip this for now.

### 2. On paper or using markdown, use your knowledge of calculus to compute explicit formulas for the Greeks we discussed

### 3. Add columns of the Greeks using your formulae

### 4. Using the Newton Raphson method discussed on 9/17/25, calculate implied volatilies for each option and add this as a column to the data

### 5. Now plot the following and keep an eye out for specific relationships. We will talk about what we notice.

### 5.1 Using the ATM Strike and fixed date, plot the following:
* Call delta vs. time to maturity
* Call gamma vs. time to maturity
* Call theta vs. time to maturity
* Call vega vs. time to maturity
* Call implied volatility vs. time to maturity

Do the same for puts. What do you notice about the greeks for calls and puts?

### 5.2 From the span of 2015-2020:
* Calculate a 5,21,63,129 rolling realiezd volatilities for SPX
* Plot the implied volatilties for ATM options expiring at roughly the same 5,21,63,129 date marks. What I mean by this is, iterate through the data and calculate the implied volatility for ATM calls expriing in those times for every single day. You won't be using the same option for the 5 years if you get what I mean... Maybe it's clear already, and I'm being dramatic.
* Make some plots of IV - RV. What do you notice? When does the graph become positive?

### 5.3 Pick a date and maturity of your own choice and plot the following:
* Strike vs. delta
* Strike vs. theta
* Strike vs. IV
* Strike vs. option price

Feel free to do any additional analysis with the data at any time by the way. We are getting familiar with how options work here!

## Your work starts here

In [2]:
import numpy as np
import yfinance as yf
import os
import pandas as pd

SPX = yf.Ticker("^GSPC")
df_SPX = SPX.history(period="30y", interval="1d")
df_SPX.index = df_SPX.index.tz_localize(None)
close_series = df_SPX['Close'].round(3)

folder = r"Folder Location"

drop_cols = ['ImpliedVolatility', 'Delta','Gamma', 'Vega', 'Theta']

for file in os.listdir(folder):
    if file.endswith(".pkl"):
        file_path = os.path.join(folder, file)

        df = pd.read_pickle(file_path)
        header_str = df.columns[0]
        new_columns = header_str.split('\t')

        df = df.iloc[:,0].str.split("\t", expand=True)
        df.columns = new_columns
        df["Date"] = pd.to_datetime(df["Date"].astype(str), format = "%Y%m%d")
        #print(df.head())
        df = df.drop(drop_cols, axis=1, errors='ignore')
        df.set_index("Date", inplace=True)
        df["SPX"] = close_series.reindex(df.index)

        df.to_pickle(file_path)

In [3]:
print(df.columns)

Index(['SecurityID', 'symbol', 'symbolflag', 'Strike', 'Expiration', 'CallPut',
       'BestBid', 'BestOffer', 'LastTradeDate', 'Volume', 'OpenInterest',
       'SpecialSettlement', 'OptionID2', 'AdjustmentFactor', 'AMSettlement',
       'ContractSize', 'ExpiryIndicator', 'SPX'],
      dtype='object')


In [5]:
import numpy as np
from scipy.stats import norm
import pandas as pd
import os

r= 0.04
sigma = 0.4

folder = r"Folder Location"

def delta(S, K, T, r, sigma, option_type = 'C'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    if option_type == 'C':
        return norm.cdf(d1)
    else:
        return norm.cdf(d1) - 1

for file in os.listdir(folder):
    if file.endswith(".pkl"):
        file_path = os.path.join(folder, file)

        df = pd.read_pickle(file_path)

        df["SPX"] = pd.to_numeric(df["SPX"], errors="coerce")
        df["Strike"] = pd.to_numeric(df["Strike"], errors="coerce")
        df["Strike"] = df["Strike"] / 1000

        df["Expiration"] = pd.to_datetime(df["Expiration"], format="%Y%m%d")
        df["T"] = (df["Expiration"] - df.index).dt.days / 365
        df["Delta"] = df.apply(lambda row: delta(S=row["SPX"], K = row["Strike"], T = max(row["T"], 0), r = r, sigma = sigma, option_type = row["CallPut"]), axis = 1)
        print(df.head())

         #df.to_pickle(file_path)

           SecurityID    symbol symbolflag  Strike Expiration CallPut BestBid  \
Date                                                                            
1996-01-04     108105  098A3.1A          0   600.0 1996-03-16       C   24.75   
1996-01-05     108105  098A3.1A          0   600.0 1996-03-16       C  24.625   
1996-01-08     108105  098A3.1A          0   600.0 1996-03-16       C    26.5   
1996-01-09     108105  098A3.1A          0   600.0 1996-03-16       C  17.375   
1996-01-10     108105  098A3.1A          0   600.0 1996-03-16       C   14.25   

           BestOffer LastTradeDate Volume OpenInterest SpecialSettlement  \
Date                                                                       
1996-01-04     25.75      19960104    150         5633                 0   
1996-01-05    25.375      19960105     50         5648                 0   
1996-01-08      27.5      19960105      0         5648                 0   
1996-01-09    18.125      19960109      1         56

KeyboardInterrupt: 